In [8]:
import pandas as pd
import utm

In [3]:
#Function to convert geographic to projected coordinates
def from_latlon(lat, lon):
    easting, northing, zone_num, zone_letter = utm.from_latlon(lat, lon)
    return easting, northing, zone_num, zone_letter

In [10]:
def get_sim_data(data, status, freq, RT_XY):
    # Get data
    RTdat_filt = RTdat[RTdat['Status'] == status]

    # make column with the datetime to nearest 'freq' value (e.g. 5min)
    RTdat_filt = RTdat_filt.assign(DateTime = RTdat_filt['DateAndTime'].dt.floor(freq=freq))

    # group by datetime, tag, tower and antenna, compute mean power, pivot to antennas
    RTdat_filt = (
        RTdat_filt.groupby(['DateTime', 'TowerID', 'TagID', 'Antenna', 'POINT_X', 'POINT_Y'])['Power']
        .mean()
        .reset_index()
        .pivot_table(index=['DateTime', 'TowerID', 'TagID', 'POINT_X', 'POINT_Y'], columns='Antenna', values='Power')
        .reset_index()
        .rename(columns={1: 'ant1', 2: 'ant2', 3: 'ant3', 4: 'ant4'})
        .fillna(value=0) # Need to check the impact of this
    )
    # Calculate easting and northing from lat long
    RTdat_filt['easting'], RTdat_filt['northing'], RTdat_filt['zone_num'], RTdat_filt['zone_letter'] = from_latlon(RTdat_filt['POINT_Y'].values, RTdat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = RT_XY.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    RTdat_filt['xOffset'] = RTdat_filt['easting'] - RTdat_filt['TowerID'].map(point_x).fillna(0)
    RTdat_filt['yOffset'] = RTdat_filt['northing'] - RTdat_filt['TowerID'].map(point_y).fillna(0)

    return RTdat_filt

In [13]:
sim_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Simul_data\RTdat_sim_only_20230202.xlsx'
output_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Simul_data\RTdat_filt_standarised_XY.xlsx'

RTdat = pd.read_excel(sim_path)
RTdat['DateAndTime'] = pd.to_datetime(RTdat['DateAndTime'])

#Get tower locations
rt_xy_path = 'H:\My Drive\Colab Notebooks\RadioTelemetry\Tower_data\RTEastNorth.xlsx'
RT_XY = pd.read_excel(rt_xy_path)

status = 'Deployed'
freq = '5min' # in Pandas frequency aliases format
data = RTdat

RTdatfilt = get_sim_data(data, status, freq, RT_XY)

In [6]:
RTdat_filt.to_excel(output_path, index=False)

In [14]:
RTdat_filt

Antenna,DateTime,TowerID,TagID,POINT_X,POINT_Y,ant1,ant2,ant3,ant4,easting,northing,zone_num,zone_letter,xOffset,yOffset
0,2021-02-02 07:50:00,RT01,60,146.256427,-21.919968,0.0,123.000000,0.0,133.0,423203.210195,7.575845e+06,55,K,-135.520989,-73.105345
1,2021-02-02 07:50:00,RT01,60,146.256751,-21.919946,0.0,122.000000,0.0,127.5,423236.662733,7.575848e+06,55,K,-102.068451,-70.508028
2,2021-02-02 07:50:00,RT04,60,146.256427,-21.919968,116.0,0.000000,127.5,0.0,423203.210195,7.575845e+06,55,K,257.092334,171.458690
3,2021-02-02 07:50:00,RT04,60,146.256751,-21.919946,114.5,0.000000,0.0,0.0,423236.662733,7.575848e+06,55,K,290.544872,174.056007
4,2021-02-02 07:55:00,RT01,60,146.256427,-21.919968,0.0,118.000000,0.0,126.5,423203.210195,7.575845e+06,55,K,-135.520989,-73.105345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,2022-02-23 07:50:00,RT18,125,146.241108,-21.932803,47.0,57.666667,0.0,0.0,421628.012884,7.574417e+06,55,K,498.275067,429.873031
155,2022-02-23 07:55:00,RT18,125,146.241108,-21.932803,63.0,65.000000,0.0,0.0,421628.012884,7.574417e+06,55,K,498.275067,429.873031
156,2022-02-23 09:30:00,RT18,126,146.240347,-21.934927,0.0,59.000000,0.0,0.0,421550.584313,7.574181e+06,55,K,420.846496,194.373856
157,2022-02-23 09:35:00,RT18,126,146.240347,-21.934927,24.0,79.200000,0.0,0.0,421550.584313,7.574181e+06,55,K,420.846496,194.373856
